In [1]:
import pandas as pd

<font size=4 color='4DA1A9'>The outline of these notebooks were influenced by Phil Schmid's [blog post](https://www.philschmid.de/fine-tune-modern-bert-in-2025) </font>

In [ ]:
# !pip install git+https://github.com/huggingface/transformers.git

<font size=4> Loading The Data </font>

In [2]:
column_names=[
        'number',
        'id',                # Column 1: the ID of the statement ([ID].json).
        'label',             # Column 2: the label.
        'statement',         # Column 3: the statement.
        'subjects',          # Column 4: the subject(s).
        'speaker',           # Column 5: the speaker.
        'speaker_job_title', # Column 6: the speaker's job title.
        'state_info',        # Column 7: the state info.
        'party_affiliation', # Column 8: the party affiliation.
        
        # Column 9-13: the total credit history count, including the current statement.
        'count_1', # barely true counts.
        'count_2', # false counts.
        'count_3', # half true counts.
        'count_4', # mostly true counts.
        'count_5', # pants on fire counts.

        'context', # Column 14: the context (venue / location of the speech or statement).
        'justification'
]
train_data = pd.read_csv('./liar_plus/train2.tsv', sep='\t', header=None, names=column_names)
test_data  = pd.read_csv('./liar_plus/test2.tsv',  sep='\t', header=None, names=column_names)
valid_data = pd.read_csv('./liar_plus/val2.tsv', sep='\t', header=None, names=column_names)

<font size=4> Preprocessing the Data </font>

In [3]:
from datasets import Dataset

def preprocess_data(data:pd.DataFrame, six_way:bool=True, metadata:bool=True):

    # these data are not usable
    data.drop(columns=[f'count_{i+1}' for i in range(5)], inplace=True)

    # encoding output labels 
    if six_way:
        numerical={'pants-fire': 0, 'false': 1, 'barely-true': 2, 'half-true': 3, 'mostly-true': 4, 'true': 5}
        data['label'] = data['label'].map(numerical)
    else:
        true_labels= ['true', 'mostly-true', 'half-true']
        data['label'] = data['label'].apply(lambda x: 1 if x in true_labels else 0)

    # fill missing columns
    data.fillna('',inplace=True)

    # adding metadata
    if metadata:
        data['statement'] = data['statement'] +' ' + data['justification'] + ' ' + data['speaker'] + ' ' + data['speaker_job_title'] + ' ' + data['state_info'] + ' ' + data['party_affiliation'] + ' ' + data['context']

    # dropping every column other than label and statement
    data.drop(columns=data.columns.difference(['label', 'statement']), inplace=True)

    # data_dict= data.to_dict(orient='records')
    # return data_dict
    return Dataset.from_pandas(data)


In [4]:
train_data= preprocess_data(train_data)
test_data= preprocess_data(test_data)
valid_data= preprocess_data(valid_data)

<font size=4> I like it better when it is a single dataset </font>

In [5]:
from datasets import dataset_dict

dataset_dict= dataset_dict.DatasetDict()

dataset_dict['train']= train_data
dataset_dict['test']= test_data
dataset_dict['valid']= valid_data

In [6]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['label', 'statement'],
        num_rows: 10240
    })
    test: Dataset({
        features: ['label', 'statement'],
        num_rows: 1267
    })
    valid: Dataset({
        features: ['label', 'statement'],
        num_rows: 1284
    })
})

In [7]:
dataset_dict['train'][0]

{'label': 1,
 'statement': "Says the Annies List political group supports third-trimester abortions on demand. That's a premise that he fails to back up. Annie's List makes no bones about being comfortable with candidates who oppose further restrictions on late-term abortions. Then again, this year its backing two House candidates who voted for more limits. dwayne-bohac State representative Texas republican a mailer"}

### Tokenizer

In [17]:
from transformers import AutoTokenizer
import torch

# model_id = "answerdotai/ModernBERT-base"
model_id = "google-bert/bert-base-uncased"
# model_id='distilbert/distilbert-base-uncased'

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.model_max_length=512


# Tokenize function
def tokenize(sample):
    return tokenizer(sample['statement'], padding='max_length', truncation=True, return_tensors="pt")

In [18]:
# Tokenize the data

tokenized_dataset= dataset_dict.map(tokenize, batched=True, remove_columns=['statement'])

Map:   0%|          | 0/10240 [00:00<?, ? examples/s]

Map:   0%|          | 0/1267 [00:00<?, ? examples/s]

Map:   0%|          | 0/1284 [00:00<?, ? examples/s]

In [19]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 10240
    })
    test: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1267
    })
    valid: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1284
    })
})

### Classifier

In [ ]:
from transformers import AutoModelForSequenceClassification

# model_id = "answerdotai/ModernBERT-base"
# model_id='distilbert/distilbert-base-uncased'
model_id = "google-bert/bert-base-uncased"


labels_2_id={'pants-fire': 0, 'false': 1, 'barely-true': 2, 'half-true': 3, 'mostly-true': 4, 'true': 5}
id_2_labels={v:k for k,v in labels_2_id.items()}


model=AutoModelForSequenceClassification.from_pretrained(
    model_id, torch_dtype=torch.bfloat16, label2id=labels_2_id, id2label=id_2_labels, num_labels=len(id_2_labels)
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
# Setting the model to run on device

device = "cuda:0" if torch.cuda.is_available() else "cpu"

model=model.to(device)

In [22]:
import numpy as np
from sklearn.metrics import f1_score, precision_recall_fscore_support, accuracy_score
 
# Metric helper method
def compute_metrics(pred):
    logits, labels = pred
    predictions = np.argmax(logits, axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="weighted")
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}


In [23]:
from transformers import Trainer, TrainingArguments
 
# Define training args
training_args = TrainingArguments(
    output_dir= "modernbert",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=1e-4,
    num_train_epochs=3,
    bf16=True, # bfloat16 training 
    optim="adamw_torch_fused", # improved optimizer 
    weight_decay=0.01,
    warmup_ratio=0.1,
    # logging & evaluation strategies
    logging_strategy="steps",
    logging_steps=100,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)
 
# Create a Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['valid'],
    compute_metrics=compute_metrics,
)

In [24]:
trainer.train()
# results=trainer.evaluate(tokenized_dataset['test'])
# results
# accuracy

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,1.698300,1.690390,0.270249,0.216205,0.287886,0.270249
2,1.642000,1.649411,0.280374,0.271745,0.296416,0.280374
3,1.478100,1.679616,0.282710,0.269713,0.295698,0.282710


/cta/users/kguray/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


TrainOutput(global_step=1920, training_loss=1.6362003962198892, metrics={'train_runtime': 283.4057, 'train_samples_per_second': 108.396, 'train_steps_per_second': 6.775, 'total_flos': 8083061908439040.0, 'train_loss': 1.6362003962198892, 'epoch': 3.0})

In [25]:
trainer.evaluate(tokenized_dataset['test'])

{'eval_loss': 1.6686054468154907,
 'eval_accuracy': 0.27466456195737965,
 'eval_f1': 0.26911314063316016,
 'eval_precision': 0.2881080191728054,
 'eval_recall': 0.27466456195737965,
 'eval_runtime': 3.5136,
 'eval_samples_per_second': 360.597,
 'eval_steps_per_second': 22.769,
 'epoch': 3.0}